In [1]:
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as yh
from stock_clustering import clusters_dict
from stock_dataset import stock_data

import warnings

warnings.filterwarnings("ignore")

/Users/miladziekanowska/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Stock data ready to use!
Clustering done!


In [3]:
clusters_dict[2] = clusters_dict.pop(4)
clusters_dict.items()

dict_items([(0, ['AXP', 'BA', 'CSCO', 'DOW', 'GS', 'HD', 'HON', 'JPM', 'NKE', 'V']), (1, ['CAT', 'CVX', 'IBM', 'JNJ', 'KO', 'MCD', 'MRK', 'TRV', 'UNH']), (3, ['CRM', 'DIS', 'INTC', 'MMM', 'VZ', 'WBA']), (2, ['AMGN'])])

Teraz co ja chcę z tym zrobić?
- potrzebuję utworzyć osobne słowniki dfów dla każdego klastra - **ZROBIONE**
- ekperymentalnie, na jednym ze słowników zrobić dwa modele - dla close i dla daily change:
    - najpierw zmienić dane na windowed df dla lstm'a (długość? trzy ostatnie dni starczą chyba); **ZROBIONE**
    - rozdzielić na train val test **ZROBIONE**
    - zbudować model dla close i model dla dailychange
    - porównać modele pod kątem sprawdzania się dla różnych firm
    - jeśli źle, przemyśleć klastryzację
    - jeśli w miarę dobrze, wybrać na czym opieramy model (close czy dailychange)
- stworzyć modele dla wszystkich klastrów
- ocenić ich jakość
- zapisać modele
- sprawdzić, jak je douczać

In [7]:
stock_data['AAPL'].shape

(753, 2)

In [4]:
cluster_data = {}
for cluster, firm_names in clusters_dict.items():
    cluster_data[cluster] = {firm_name: stock_data[name] for firm_name in firm_names}

In [5]:
cluster_0 = {}
for firm_name, df in cluster_data[0].items():
    cluster_0[name] = df
    
cluster_1 = {}
for firm_name, df in cluster_data[1].items():
    cluster_1[name] = df
    
cluster_2 = {}
for firm_name, df in cluster_data[2].items():
    cluster_2[name] = df

cluster_3 = {}
for firm_name, df in cluster_data[3].items():
    cluster_3[name] = df

In [7]:
cluster_0['CSCO']

array([[-0.39092168, -0.01458432],
       [-0.38223968,  0.05461382],
       [-0.40307661, -0.18048431],
       ...,
       [ 0.40088827,  0.99752315],
       [ 0.20119965, -1.46532054],
       [ 0.27586603,  0.54024857]])

In [8]:
closing = {}
for key in cluster_0.keys():
    closing[key] = cluster_0[key][:,0]

In [10]:
daily_change = {}
for key in cluster_0.keys():
    daily_change[key] = cluster_0[key][:,1]

In [19]:
window = 4

X = {}
Y = {}

for key in closing.keys():
    for i in range(window, len(closing[key])):
        X[key].append(closing[key][i-window:i])
        Y[key].append(closing[key][i])

In [24]:
q_80 = int(len(X['CSCO']) * .8)
q_99 = int(len(X['CSCO']) * .99)

X_train, X_val, X_test = {}, {}, {}
Y_train, Y_val, Y_test = {}, {}, {}

for key in X.keys():
    X_train[key], X_val[key], X_test[key] = X[key][:q_80], X[key][q_80:q_99], X[key][q_99:]

for key in Y.keys():
    Y_train[key], Y_val[key], Y_test[key] = Y[key][:q_80], Y[key][q_80:q_99], Y[key][q_99:]                                                


[array([-0.39092168, -0.38223968, -0.40307661, -0.31451916]),
 array([-0.38223968, -0.40307661, -0.31451916, -0.48989793]),
 array([-0.40307661, -0.31451916, -0.48989793, -0.35272075]),
 array([-0.31451916, -0.48989793, -0.35272075, -0.51073552]),
 array([-0.48989793, -0.35272075, -0.51073552, -0.43606914]),
 array([-0.35272075, -0.51073552, -0.43606914, -0.48295272]),
 array([-0.51073552, -0.43606914, -0.48295272, -0.36834862]),
 array([-0.43606914, -0.48295272, -0.36834862, -0.35792982]),
 array([-0.48295272, -0.36834862, -0.35792982, -0.26937237]),
 array([-0.36834862, -0.35792982, -0.26937237, -0.32841023]),
 array([-0.35792982, -0.26937237, -0.32841023, -0.25200771]),
 array([-0.26937237, -0.32841023, -0.25200771, -0.31104623]),
 array([-0.32841023, -0.25200771, -0.31104623, -0.25895358]),
 array([-0.25200771, -0.31104623, -0.25895358, -0.35272075]),
 array([-0.31104623, -0.25895358, -0.35272075, -0.19470599]),
 array([-0.25895358, -0.35272075, -0.19470599, -1.12890285]),
 array([

In [28]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [29]:
model=Sequential()
model.add(LSTM(50, activation='relu', input_shape=(4,1)))
model.add(Dense(25))
model.add(Dense(1))

In [30]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 25)                1275      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 11701 (45.71 KB)
Trainable params: 11701 (45.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
for key in X.keys():
    model.fit(X_train[key], Y_train[key], epochs=10, batch_size=10)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'list'> containing values of types {"<class 'numpy.float64'>"})